Assuming we already have SAT/AP scores and the high schools distances in the file, we would only look for high schools which are in California and we want to filter those schools out first. 

In [1]:
import pandas as pd
%matplotlib inline
import pylab as plt
import numpy as np
import scipy as sc
from scipy import stats

In [2]:
data = pd.read_csv('data/processed.csv')
data.head()

,campus,year,school,school_num,city,county,state,country,region,ethnicity,app_num,adm_num,enr_num,app_gpa,adm_gpa,enr_gpa
0,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,51520,Los Angeles,Los Angeles,California,USA,Los Angeles,All,14.0,NaN,NaN,3.620000,NaN,NaN
1,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,51520,Los Angeles,Los Angeles,California,USA,Los Angeles,Asian,8.0,NaN,NaN,3.620000,NaN,NaN
2,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,51520,Los Angeles,Los Angeles,California,USA,Los Angeles,Hispanic/ Latino,5.0,NaN,NaN,3.620000,NaN,NaN
3,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,52910,San Francisco,San Francisco,California,USA,San Francisco,All,58.0,8.0,7.0,3.682931,4.12125,4.088571
4,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,52910,San Francisco,San Francisco,California,USA,San Francisco,Asian,50.0,8.0,7.0,3.682931,4.12125,4.088571


check the total number of high schools in our file 

In [13]:
tot_high = len(data['school'].unique())
tot_high

2722

In [14]:
data[data['state']=='California']
data.head()

,campus,year,school,school_num,city,county,state,country,region,ethnicity,app_num,adm_num,enr_num,app_gpa,adm_gpa,enr_gpa
0,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,51520,Los Angeles,Los Angeles,California,USA,Los Angeles,All,14.0,NaN,NaN,3.620000,NaN,NaN
1,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,51520,Los Angeles,Los Angeles,California,USA,Los Angeles,Asian,8.0,NaN,NaN,3.620000,NaN,NaN
2,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,51520,Los Angeles,Los Angeles,California,USA,Los Angeles,Hispanic/ Latino,5.0,NaN,NaN,3.620000,NaN,NaN
3,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,52910,San Francisco,San Francisco,California,USA,San Francisco,All,58.0,8.0,7.0,3.682931,4.12125,4.088571
4,Berkeley,1994,ABRAHAM LINCOLN HIGH SCHOOL,52910,San Francisco,San Francisco,California,USA,San Francisco,Asian,50.0,8.0,7.0,3.682931,4.12125,4.088571


check the total number of high schools in california 

In [48]:
num_cali_high = len(data[data['state']=='California']['school'].unique())
num_cali_high

860

In [18]:
data_cali_high = data[data['state']=='California']
data_cali_high['yield'] = data_cali_high['enr_num'] / data_cali_high['adm_num']
data_cali_high.describe()

/Users/Nelson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,year,school_num,app_num,adm_num,enr_num,app_gpa,adm_gpa,enr_gpa,yield
count,286372.000000,286372.000000,286372.000000,216242.000000,86097.000000,286372.000000,216242.000000,86097.000000,86097.000000
mean,2007.470266,52071.744570,24.500384,16.100018,11.901158,3.668796,3.885036,3.829268,0.477578
std,6.659790,16974.293167,31.383948,21.435905,15.477301,0.193622,0.226210,0.227325,0.200997
min,1994.000000,50003.000000,5.000000,3.000000,3.000000,2.468333,2.798462,2.782000,0.034483
25%,2002.000000,50729.000000,8.000000,5.000000,5.000000,3.544737,3.722051,3.687143,0.318841
50%,2008.000000,51370.000000,14.000000,9.000000,7.000000,3.681833,3.893607,3.840000,0.466667
75%,2013.000000,52773.000000,28.000000,18.000000,13.000000,3.806000,4.066923,3.991429,0.611111
max,2017.000000,998463.000000,544.000000,472.000000,295.000000,4.356667,4.495000,4.430000,1.000000


check how many NaN's are there

In [68]:
count_nan = len(data_cali_high) - data_cali_high.count()
count_nan

campus        0
year          0
school        0
school_num    0
city          0
county        0
state         0
country       0
region        0
ethnicity     0
app_num       0
adm_num       0
enr_num       0
app_gpa       0
adm_gpa       0
enr_gpa       0
yield         0
dtype: int64

filter out the NaN data

In [69]:
#data_cali_high = data_cali_high.dropna()

X is to be fixed, it contains the features we want to include

In [70]:
X = data_cali_high[['app_num', 'adm_num', 'app_gpa', 'adm_gpa', 'year']]
y = data_cali_high['yield']

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [72]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [73]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns = ['coefficient'])

In [74]:
coeff_df

,coefficient
app_num,-0.001824
adm_num,0.001106
app_gpa,0.002504
adm_gpa,0.107427
year,-0.000636


In [75]:
y_pred = regressor.predict(X_test)

In [76]:
data_result = pd.DataFrame({'Actual Yield': y_test, 'Predicted Yield': y_pred})  
data_result.head(10)

,Actual Yield,Predicted Yield
110604,0.750000,0.550305
11204,0.625000,0.519715
281965,0.681818,0.497919
81355,0.272727,0.511098
283675,0.480000,0.485205
111205,0.750000,0.542085
337533,0.397849,0.434670
83323,0.157895,0.490841
335364,0.538462,0.520665
217707,0.750000,0.535752
